In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

In [2]:
week_nums = [200104, 200111, 200118, 200125, 200201, 200208, 200215, 200222, 200229, 200307, 200314, 200321, 200328, 200404, 200411, 200418, 200425, 200502, 200509, 200516, 200523, 200530, 200606, 200613, 200620, 200627]
df = get_data(week_nums)
# df.STATION.unique()
df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,03:00:00,REGULAR,7324295,2482512
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,07:00:00,REGULAR,7324305,2482523
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,11:00:00,REGULAR,7324371,2482594
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,15:00:00,REGULAR,7324587,2482647
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,19:00:00,REGULAR,7324963,2482713
...,...,...,...,...,...,...,...,...,...,...,...
206666,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/26/2020,05:00:00,REGULAR,5554,514
206667,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/26/2020,09:00:00,REGULAR,5554,514
206668,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/26/2020,13:00:00,REGULAR,5554,514
206669,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/26/2020,17:00:00,REGULAR,5554,514


## By station: template

### TIMES SQ-42 ST (no groupby linename)

In [120]:
##### TIMES SQ-42 ST (no groupby linename)#####

df_42_timesquare = df[df.STATION == 'TIMES SQ-42 ST']

# df_42_timesquare['DATETIME'] = df_42_timesquare['DATE'] + " " + df_42_timesquare['TIME']
# df_42_timesquare['DATETIME'] = pd.to_datetime(df_42_timesquare['DATETIME'])
df_42_timesquare['DATE'] = pd.to_datetime(df_42_timesquare['DATE'])
df_42_timesquare['TIME'] = pd.to_datetime(df_42_timesquare['TIME'])
df_42_timesquare.rename(columns={'EXITS                                                               ': 'EXITS'}
                        , inplace=True)

# get the daily summary data of each turnstile
daily_sum_42_timesquare = df_42_timesquare.groupby(['C/A', 'UNIT', 'SCP', 'LINENAME', 'STATION','DATE']).agg({'ENTRIES': lambda ENTIRES : ENTIRES.max() - ENTIRES.min(),
                                                         'EXITS': lambda EXITS: EXITS.max() - EXITS.min()}).reset_index()
# group by date (I kept 'STATION' in groupby so the column would show in the dataframe); get the daily summary data of station
daily_sum_42_timesquare = daily_sum_42_timesquare.groupby(['STATION','DATE'])[['ENTRIES','EXITS']].sum().reset_index()
daily_sum_42_timesquare

,STATION,DATE,ENTRIES,EXITS
0,TIMES SQ-42 ST,2019-12-28,60813,72374
1,TIMES SQ-42 ST,2019-12-29,49246,55446
2,TIMES SQ-42 ST,2019-12-30,79168,86044
3,TIMES SQ-42 ST,2019-12-31,21093,68253
4,TIMES SQ-42 ST,2020-01-01,45001,52835
...,...,...,...,...
177,TIMES SQ-42 ST,2020-06-22,9746,11031
178,TIMES SQ-42 ST,2020-06-23,10331,11773
179,TIMES SQ-42 ST,2020-06-24,10443,11949
180,TIMES SQ-42 ST,2020-06-25,10493,12192


In [121]:
import datetime as dt
daily_sum_42_timesquare['weekday'] = daily_sum_42_timesquare['DATE'].dt.dayofweek
daily_sum_42_timesquare['WeekOf'] = daily_sum_42_timesquare['DATE']
daily_sum_42_timesquare.loc[daily_sum_42_timesquare.weekday != 0, 'WeekOf'] = ''
daily_sum_42_timesquare = daily_sum_42_timesquare.replace('',np.nan).ffill()
daily_sum_42_timesquare['WeekOf'] = pd.to_datetime(daily_sum_42_timesquare['WeekOf']).dt.date
daily_sum_42_timesquare

,STATION,DATE,ENTRIES,EXITS,weekday,WeekOf
0,TIMES SQ-42 ST,2019-12-28,60813,72374,5,NaT
1,TIMES SQ-42 ST,2019-12-29,49246,55446,6,NaT
2,TIMES SQ-42 ST,2019-12-30,79168,86044,0,2019-12-30
3,TIMES SQ-42 ST,2019-12-31,21093,68253,1,2019-12-30
4,TIMES SQ-42 ST,2020-01-01,45001,52835,2,2019-12-30
...,...,...,...,...,...,...
177,TIMES SQ-42 ST,2020-06-22,9746,11031,0,2020-06-22
178,TIMES SQ-42 ST,2020-06-23,10331,11773,1,2020-06-22
179,TIMES SQ-42 ST,2020-06-24,10443,11949,2,2020-06-22
180,TIMES SQ-42 ST,2020-06-25,10493,12192,3,2020-06-22


In [122]:
daily_sum_42_timesquare.groupby(['WeekOf','STATION'])[['ENTRIES','EXITS']].sum()

,,ENTRIES,EXITS
WeekOf,STATION,,
2019-12-30,TIMES SQ-42 ST,401608,491900
2020-01-06,TIMES SQ-42 ST,596013,586251
2020-01-13,TIMES SQ-42 ST,522531,537979
2020-01-20,TIMES SQ-42 ST,482393,500385
2020-01-27,TIMES SQ-42 ST,515892,532835
2020-02-03,TIMES SQ-42 ST,508939,529508
2020-02-10,TIMES SQ-42 ST,523360,554400
2020-02-17,TIMES SQ-42 ST,1014967,1367654
2020-02-24,TIMES SQ-42 ST,508423,539070


### GRD CNTRL-42 ST (no groupby linename)

In [133]:
##### GRD CNTRL-42 ST (no groupby linename)#####

df_42_grand = df[df.STATION == 'GRD CNTRL-42 ST']

# df_42_timesquare['DATETIME'] = df_42_timesquare['DATE'] + " " + df_42_timesquare['TIME']
# df_42_timesquare['DATETIME'] = pd.to_datetime(df_42_timesquare['DATETIME'])
df_42_grand['DATE'] = pd.to_datetime(df_42_grand['DATE'])
df_42_grand['TIME'] = pd.to_datetime(df_42_grand['TIME'])
df_42_grand.rename(columns={'EXITS                                                               ': 'EXITS'}
                        , inplace=True)

# get the daily summary data of each turnstile
daily_sum_42_grand = df_42_grand.groupby(['C/A', 'UNIT', 'SCP', 'LINENAME', 'STATION','DATE']).agg({'ENTRIES': lambda ENTIRES : ENTIRES.max() - ENTIRES.min(),
                                                         'EXITS': lambda EXITS: EXITS.max() - EXITS.min()}).reset_index()
# group by date (I kept 'STATION' in groupby so the column would show in the dataframe); get the daily summary data of station
daily_sum_42_grand = daily_sum_42_grand.groupby(['STATION','DATE'])[['ENTRIES','EXITS']].sum().reset_index()
daily_sum_42_grand

,STATION,DATE,ENTRIES,EXITS
0,GRD CNTRL-42 ST,2019-12-28,54505,55218
1,GRD CNTRL-42 ST,2019-12-29,42338,42789
2,GRD CNTRL-42 ST,2019-12-30,113443,110096
3,GRD CNTRL-42 ST,2019-12-31,93198,89665
4,GRD CNTRL-42 ST,2020-01-01,36861,42166
...,...,...,...,...
177,GRD CNTRL-42 ST,2020-06-22,13167,16657
178,GRD CNTRL-42 ST,2020-06-23,13401,17882
179,GRD CNTRL-42 ST,2020-06-24,13883,17949
180,GRD CNTRL-42 ST,2020-06-25,13762,18231


In [134]:
import datetime as dt
daily_sum_42_grand['weekday'] = daily_sum_42_grand['DATE'].dt.dayofweek
daily_sum_42_grand['WeekOf'] = daily_sum_42_grand['DATE']
daily_sum_42_grand.loc[daily_sum_42_grand.weekday != 0, 'WeekOf'] = ''
daily_sum_42_grand = daily_sum_42_grand.replace('',np.nan).ffill()
daily_sum_42_grand['WeekOf'] = pd.to_datetime(daily_sum_42_grand['WeekOf']).dt.date
daily_sum_42_grand

,STATION,DATE,ENTRIES,EXITS,weekday,WeekOf
0,GRD CNTRL-42 ST,2019-12-28,54505,55218,5,NaT
1,GRD CNTRL-42 ST,2019-12-29,42338,42789,6,NaT
2,GRD CNTRL-42 ST,2019-12-30,113443,110096,0,2019-12-30
3,GRD CNTRL-42 ST,2019-12-31,93198,89665,1,2019-12-30
4,GRD CNTRL-42 ST,2020-01-01,36861,42166,2,2019-12-30
...,...,...,...,...,...,...
177,GRD CNTRL-42 ST,2020-06-22,13167,16657,0,2020-06-22
178,GRD CNTRL-42 ST,2020-06-23,13401,17882,1,2020-06-22
179,GRD CNTRL-42 ST,2020-06-24,13883,17949,2,2020-06-22
180,GRD CNTRL-42 ST,2020-06-25,13762,18231,3,2020-06-22


In [136]:
daily_sum_42_grand.groupby(['WeekOf','STATION'])[['ENTRIES','EXITS']].sum()

,,ENTRIES,EXITS
WeekOf,STATION,,
2019-12-30,GRD CNTRL-42 ST,557946,561094
2020-01-06,GRD CNTRL-42 ST,2589189,3257667
2020-01-13,GRD CNTRL-42 ST,778931,773219
2020-01-20,GRD CNTRL-42 ST,697532,708940
2020-01-27,GRD CNTRL-42 ST,758991,777356
2020-02-03,GRD CNTRL-42 ST,1072669,1338852
2020-02-10,GRD CNTRL-42 ST,771795,795973
2020-02-17,GRD CNTRL-42 ST,635761,646551
2020-02-24,GRD CNTRL-42 ST,840039,885507


# NO GROUP BY LINENAME (7 stations) 2020

In [9]:
df_stations = df[(df.STATION == 'TIMES SQ-42 ST')|(df.STATION == 'GRD CNTRL-42 ST')|(df.STATION == 'LEXINGTON AV/53')
                 |(df.STATION == '34 ST-HERALD SQ')|(df.STATION == '14 ST-UNION SQ')|(df.STATION == '59 ST COLUMBUS')
                |(df.STATION == '34 ST-PENN STA')]

df_stations['DATE'] = pd.to_datetime(df_stations['DATE'])
df_stations['TIME'] = pd.to_datetime(df_stations['TIME'])
df_stations.rename(columns={'EXITS                                                               ': 'EXITS'}
                        , inplace=True)

# get the daily summary data of each turnstile
daily_sum_stations = df_stations.groupby(['C/A', 'UNIT', 'SCP', 'LINENAME', 'STATION','DATE']).agg({'ENTRIES': lambda ENTIRES : ENTIRES.max() - ENTIRES.min(),
                                                         'EXITS': lambda EXITS: EXITS.max() - EXITS.min()}).reset_index()
# group by date (I kept 'STATION' in groupby so the column would show in the dataframe); get the daily summary data of station
daily_sum_stations = daily_sum_stations.groupby(['STATION','DATE'])[['ENTRIES','EXITS']].sum().reset_index()
daily_sum_stations

,STATION,DATE,ENTRIES,EXITS
0,14 ST-UNION SQ,2019-12-28,45068,52230
1,14 ST-UNION SQ,2019-12-29,36281,42165
2,14 ST-UNION SQ,2019-12-30,57899,65968
3,14 ST-UNION SQ,2019-12-31,55468,59503
4,14 ST-UNION SQ,2020-01-01,30936,33736
...,...,...,...,...
1269,TIMES SQ-42 ST,2020-06-22,9746,11031
1270,TIMES SQ-42 ST,2020-06-23,10331,11773
1271,TIMES SQ-42 ST,2020-06-24,10443,11949
1272,TIMES SQ-42 ST,2020-06-25,10493,12192


In [10]:
import datetime as dt
daily_sum_stations['weekday'] = daily_sum_stations['DATE'].dt.dayofweek
daily_sum_stations['WeekOf'] = daily_sum_stations['DATE']
daily_sum_stations.loc[daily_sum_stations.weekday != 0, 'WeekOf'] = ''
daily_sum_stations = daily_sum_stations.replace('',np.nan).ffill()
daily_sum_stations['WeekOf'] = pd.to_datetime(daily_sum_stations['WeekOf']).dt.date
daily_sum_stations['TotalTraffic'] = daily_sum_stations['ENTRIES'] + daily_sum_stations['EXITS']
daily_sum_stations

,STATION,DATE,ENTRIES,EXITS,weekday,WeekOf,TotalTraffic
0,14 ST-UNION SQ,2019-12-28,45068,52230,5,NaT,97298
1,14 ST-UNION SQ,2019-12-29,36281,42165,6,NaT,78446
2,14 ST-UNION SQ,2019-12-30,57899,65968,0,2019-12-30,123867
3,14 ST-UNION SQ,2019-12-31,55468,59503,1,2019-12-30,114971
4,14 ST-UNION SQ,2020-01-01,30936,33736,2,2019-12-30,64672
...,...,...,...,...,...,...,...
1269,TIMES SQ-42 ST,2020-06-22,9746,11031,0,2020-06-22,20777
1270,TIMES SQ-42 ST,2020-06-23,10331,11773,1,2020-06-22,22104
1271,TIMES SQ-42 ST,2020-06-24,10443,11949,2,2020-06-22,22392
1272,TIMES SQ-42 ST,2020-06-25,10493,12192,3,2020-06-22,22685


## Analysis of 7 Stations for 2020

In [11]:
# weekly_stations_2020 = daily_sum_stations.groupby(['WeekOf','STATION'])[['ENTRIES','EXITS']].sum().reset_index()
# weekly_stations_2020[weekly_stations_2020.WeekOf == weekly_stations_2020.iloc[110,0]]

In [117]:
##### 34 ST-PENN STA (groupby linename)#####
df_34_penn = df[df.STATION == "34 ST-PENN STA"]

# df_34_penn['DATETIME'] = df_34_penn['DATE'] + " " + df_34_penn['TIME']
# df_34_penn['DATETIME'] = pd.to_datetime(df_34_penn['DATETIME'])
df_34_penn['DATE'] = pd.to_datetime(df_34_penn['DATE'])
df_34_penn['TIME'] = pd.to_datetime(df_34_penn['TIME'])
df_34_penn.rename(columns={'EXITS                                                               ': 'EXITS'}
                        , inplace=True)

# get the daily summary data of each turnstile
daily_sum_34_penn = df_34_penn.groupby(['C/A', 'UNIT', 'SCP', 'LINENAME', 'STATION','DATE']).agg({'ENTRIES': lambda ENTIRES : ENTIRES.max() - ENTIRES.min(),
                                                         'EXITS': lambda EXITS: EXITS.max() - EXITS.min()}).reset_index()

# group by date and linename; get the daily summary data of station
daily_sum_34_pennALL = daily_sum_34_penn.groupby(['STATION','DATE','LINENAME'])[['ENTRIES','EXITS']].sum().reset_index()
daily_sum_34_penn123 = daily_sum_34_pennALL[daily_sum_34_pennALL.LINENAME == "123"]
daily_sum_34_pennACE = daily_sum_34_pennALL[daily_sum_34_pennALL.LINENAME == "ACE"]
daily_sum_34_penn123ACE = daily_sum_34_pennALL[daily_sum_34_pennALL.LINENAME == "123ACE"]

In [5]:
daily_sum_34_penn123

,STATION,DATE,LINENAME,ENTRIES,EXITS
0,34 ST-PENN STA,2020-06-06,123,923,1896
3,34 ST-PENN STA,2020-06-07,123,800,1474
6,34 ST-PENN STA,2020-06-08,123,2371,3260
9,34 ST-PENN STA,2020-06-09,123,2305,3266
12,34 ST-PENN STA,2020-06-10,123,2461,3351
15,34 ST-PENN STA,2020-06-11,123,2354,3202
18,34 ST-PENN STA,2020-06-12,123,2391,3521
21,34 ST-PENN STA,2020-06-20,123,1264,2596
24,34 ST-PENN STA,2020-06-21,123,1032,1871
27,34 ST-PENN STA,2020-06-22,123,3324,4200


In [6]:
daily_sum_34_pennACE

,STATION,DATE,LINENAME,ENTRIES,EXITS
2,34 ST-PENN STA,2020-06-06,ACE,3234,4830
5,34 ST-PENN STA,2020-06-07,ACE,2893,3754
8,34 ST-PENN STA,2020-06-08,ACE,7134,8634
11,34 ST-PENN STA,2020-06-09,ACE,7269,9065
14,34 ST-PENN STA,2020-06-10,ACE,7506,9179
17,34 ST-PENN STA,2020-06-11,ACE,7397,9493
20,34 ST-PENN STA,2020-06-12,ACE,7627,9936
23,34 ST-PENN STA,2020-06-20,ACE,3980,6284
26,34 ST-PENN STA,2020-06-21,ACE,3446,4951
29,34 ST-PENN STA,2020-06-22,ACE,9806,11552


In [7]:
daily_sum_34_penn123ACE

,STATION,DATE,LINENAME,ENTRIES,EXITS
1,34 ST-PENN STA,2020-06-06,123ACE,2455,3056
4,34 ST-PENN STA,2020-06-07,123ACE,2197,2569
7,34 ST-PENN STA,2020-06-08,123ACE,4646,5129
10,34 ST-PENN STA,2020-06-09,123ACE,4646,5314
13,34 ST-PENN STA,2020-06-10,123ACE,4719,5575
16,34 ST-PENN STA,2020-06-11,123ACE,4568,5453
19,34 ST-PENN STA,2020-06-12,123ACE,4928,6014
22,34 ST-PENN STA,2020-06-20,123ACE,3089,4038
25,34 ST-PENN STA,2020-06-21,123ACE,2857,2989
28,34 ST-PENN STA,2020-06-22,123ACE,6162,6777


## Not by station:

In [123]:
df['DATETIME'] = df['DATE'] + " " + df['TIME']
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['DATE'] = pd.to_datetime(df['DATE'])
df['TIME'] = pd.to_datetime(df['TIME'])
df.rename(columns={'EXITS                                                               ': 'EXITS'}, inplace=True)

In [124]:
# get the daily summary data of each turnstile
daily_sum = df.groupby(['C/A', 'UNIT', 'SCP', 'LINENAME', 'STATION','DATE']).agg({'ENTRIES': lambda ENTIRES : ENTIRES.max() - ENTIRES.min(),
                                                          'EXITS': lambda EXITS: EXITS.max() - EXITS.min()}).reset_index()

In [132]:
daily_sum[(daily_sum.DATE == '2020-05-04') & (daily_sum.STATION == 'GRD CNTRL-42 ST')].groupby(['STATION']).sum()

,ENTRIES,EXITS
STATION,,
GRD CNTRL-42 ST,5616,7638


In [12]:
# check if the count was reset to 0
daily_sum[daily_sum.ENTRIES < 0]

,C/A,UNIT,SCP,LINENAME,STATION,DATE,ENTRIES,EXITS


In [13]:
daily_sum[daily_sum.EXITS < 0]

,C/A,UNIT,SCP,LINENAME,STATION,DATE,ENTRIES,EXITS


In [14]:
# group by station, linename and date
daily_sum.groupby(['STATION','DATE', 'LINENAME'])[['ENTRIES','EXITS']].sum().reset_index()

,STATION,DATE,LINENAME,ENTRIES,EXITS
0,1 AV,2020-06-06,L,1451,2422
1,1 AV,2020-06-07,L,1095,1998
2,1 AV,2020-06-08,L,2349,3683
3,1 AV,2020-06-09,L,2503,3939
4,1 AV,2020-06-10,L,2609,4177
...,...,...,...,...,...
6701,ZEREGA AV,2020-06-22,6,666,741
6702,ZEREGA AV,2020-06-23,6,671,702
6703,ZEREGA AV,2020-06-24,6,613,700
6704,ZEREGA AV,2020-06-25,6,579,668
